In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
import numpy as np
from math import sqrt

In [2]:
data_classifier = pd.read_csv("D:\ML\data\weather_data.csv", delimiter=',')
data_regressor = pd.read_csv("D:\ML\data\data.csv", delimiter=',')

scaler = MinMaxScaler(feature_range=(0,1))
scaler.fit(data_classifier)

data_classifier = scaler.transform(data_classifier)
data_classifier = pd.DataFrame(data_classifier)

data_classifier = data_classifier[:1000]

balance = len(data_classifier.loc[data_classifier[16]==0])//len(data_classifier.loc[data_classifier[16]==1])

data_sample = data_classifier.loc[data_classifier[16]==1]
data_sample = data_sample.loc[data_sample.index.repeat(balance)]
data_n = pd.concat([data_classifier.loc[data_classifier[16]==0], data_sample]).sample(frac=1)


y_regression = data_regressor["quality"]
X_regression = data_regressor.drop(["quality"], axis=1)
y_classification = data_n[16]
X_classification = data_n.drop([16], axis=1)

data_classifier.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Columns: 119 entries, 0 to 118
dtypes: float64(119)
memory usage: 929.8 KB


In [3]:
data_regressor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5320 entries, 0 to 5319
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         5320 non-null   float64
 1   volatile acidity      5320 non-null   float64
 2   citric acid           5320 non-null   float64
 3   residual sugar        5320 non-null   float64
 4   chlorides             5320 non-null   float64
 5   free sulfur dioxide   5320 non-null   float64
 6   total sulfur dioxide  5320 non-null   float64
 7   density               5320 non-null   float64
 8   pH                    5320 non-null   float64
 9   sulphates             5320 non-null   float64
 10  alcohol               5320 non-null   float64
 11  quality               5320 non-null   float64
 12  kind_red              5320 non-null   float64
 13  kind_white            5320 non-null   float64
dtypes: float64(14)
memory usage: 582.0 KB


In [4]:
from sklearn.model_selection import train_test_split
X_regression_train, X_regression_test, y_regression_train, y_regression_test = train_test_split(X_regression,
                                                                                                y_regression,
                                                                                                test_size=0.2)
X_classification_train, X_classification_test, y_classification_train, y_classification_test = train_test_split(X_classification,
                                                                                                                y_classification,
                                                                                                                stratify=y_classification,
                                                                                                                test_size=0.2)

In [5]:
model_regression = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(64, activation="relu", input_shape=(13,)),
        tf.keras.layers.Dense(32, activation="linear"),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(1, activation="linear"),
    ]
)

In [7]:
model_regression.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                896       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 16)                528       
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 3,521
Trainable params: 3,521
Non-trainabl

In [8]:
model_regression.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), loss="mse")

In [9]:
model_regression.fit(X_regression_train, y_regression_train, epochs=50)

Epoch 1/50
133/133 [==============================] - 2s 3ms/step - loss: 15.7599
Epoch 2/50
133/133 [==============================] - 0s 2ms/step - loss: 2.5697
Epoch 3/50
133/133 [==============================] - 0s 2ms/step - loss: 2.1163
Epoch 4/50
133/133 [==============================] - 0s 2ms/step - loss: 1.9202
Epoch 5/50
133/133 [==============================] - 0s 2ms/step - loss: 1.6184
Epoch 6/50
133/133 [==============================] - 0s 2ms/step - loss: 1.4579
Epoch 7/50
133/133 [==============================] - 0s 2ms/step - loss: 1.3459
Epoch 8/50
133/133 [==============================] - 0s 2ms/step - loss: 1.2923
Epoch 9/50
133/133 [==============================] - 0s 2ms/step - loss: 1.2266
Epoch 10/50
133/133 [==============================] - 0s 2ms/step - loss: 1.2253
Epoch 11/50
133/133 [==============================] - 0s 2ms/step - loss: 1.1846
Epoch 12/50
133/133 [==============================] - 0s 2ms/step - loss: 1.0639
Epoch 13/50
133/133 [===

In [10]:
print(mean_absolute_error(y_regression_test, model_regression.predict(X_regression_test)))
print(mean_squared_error(y_regression_test, model_regression.predict(X_regression_test)))
print(mean_absolute_percentage_error(y_regression_test, model_regression.predict(X_regression_test)))
print(sqrt(mean_squared_error(y_regression_test, model_regression.predict(X_regression_test))))

34/34 [==============================] - 0s 2ms/step
0.610996034808625
34/34 [==============================] - 0s 2ms/step
0.5980170786809164
34/34 [==============================] - 0s 2ms/step
0.11285227356612619
34/34 [==============================] - 0s 2ms/step
0.7733156397493305


In [11]:
model_classification_1 = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(64, activation="relu", input_shape=(118,)),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.05),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ]
)

model_classification_1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss="mse")
model_classification_1.fit(X_classification_train, y_classification_train, epochs=25, verbose=None)

In [12]:
model_classification_1.predict(X_classification_test, verbose=None)[:5]

array([[0.9598435 ],
       [0.99116   ],
       [0.9394521 ],
       [0.98863643],
       [0.94279975]], dtype=float32)

In [13]:
y_pred = np.around(model_classification_1.predict(X_classification_test, verbose=None))
print(classification_report(y_classification_test, y_pred))
print(confusion_matrix(y_classification_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.93      0.88      0.90      1621
         1.0       0.88      0.93      0.90      1519

    accuracy                           0.90      3140
   macro avg       0.90      0.90      0.90      3140
weighted avg       0.91      0.90      0.90      3140

[[1419  202]
 [ 101 1418]]


In [14]:
w0 = 1 / y_classification_train[y_classification_train==0].shape[0]
w1 = 1 / y_classification_train[y_classification_train==1].shape[0]

In [15]:
model_classification_1 = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(64, activation="relu", input_shape=(118,)),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.05),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ]
)
model_classification_1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), loss="binary_crossentropy")
model_classification_1.fit(X_classification_train, y_classification_train, epochs=25, verbose=None,
                           class_weight={0: w0, 1: w1})
y_pred = np.around(model_classification_1.predict(X_classification_test, verbose=None))
print(classification_report(y_classification_test, y_pred))
print(confusion_matrix(y_classification_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.96      0.85      0.90      1621
         1.0       0.86      0.96      0.91      1519

    accuracy                           0.90      3140
   macro avg       0.91      0.91      0.90      3140
weighted avg       0.91      0.90      0.90      3140

[[1383  238]
 [  63 1456]]


In [16]:
model_classification_2 = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(64, activation="relu", input_shape=(118,)),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.05),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dense(2, activation="softmax"),
    ]
)
model_classification_2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), loss="sparse_categorical_crossentropy")
model_classification_2.fit(X_classification_train, y_classification_train, epochs=25, verbose=None,
                           class_weight={0: w0, 1: w1})

In [17]:
model_classification_2.predict(X_classification_test, verbose=None)[:5]

array([[0.03064739, 0.9693526 ],
       [0.03000836, 0.9699916 ],
       [0.16266857, 0.8373314 ],
       [0.00269466, 0.9973053 ],
       [0.21390928, 0.7860907 ]], dtype=float32)

In [18]:
y_pred = [np.argmax(pred) for pred in model_classification_2.predict(X_classification_test, verbose=None)]

In [19]:
print(classification_report(y_classification_test, y_pred))
print(confusion_matrix(y_classification_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.95      0.85      0.90      1621
         1.0       0.86      0.96      0.90      1519

    accuracy                           0.90      3140
   macro avg       0.91      0.90      0.90      3140
weighted avg       0.91      0.90      0.90      3140

[[1381  240]
 [  67 1452]]


In [20]:
model_regression.save('../models/RegressionModel')
model_classification_1.save('../models/ClassificationModel1')
model_classification_2.save('../models/ClassificationModel2')

INFO:tensorflow:Assets written to: ../models/RegressionModel\assets


INFO:tensorflow:Assets written to: ../models/RegressionModel\assets


INFO:tensorflow:Assets written to: ../models/ClassificationModel1\assets


INFO:tensorflow:Assets written to: ../models/ClassificationModel1\assets


INFO:tensorflow:Assets written to: ../models/ClassificationModel2\assets


INFO:tensorflow:Assets written to: ../models/ClassificationModel2\assets


In [21]:
model_regression_restored = tf.keras.models.load_model('../models/RegressionModel')
model_regression_restored.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                896       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 16)                528       
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 3,521
Trainable params: 3,521
Non-trainabl

In [22]:
print(mean_absolute_error(y_regression_test, model_regression.predict(X_regression_test)))
print(mean_squared_error(y_regression_test, model_regression.predict(X_regression_test)))
print(mean_absolute_percentage_error(y_regression_test, model_regression.predict(X_regression_test)))
print(sqrt(mean_squared_error(y_regression_test, model_regression.predict(X_regression_test))))

34/34 [==============================] - 0s 2ms/step
0.610996034808625
34/34 [==============================] - 0s 2ms/step
0.5980170786809164
34/34 [==============================] - 0s 2ms/step
0.11285227356612619
34/34 [==============================] - 0s 2ms/step
0.7733156397493305


In [6]:
import numpy as np

class Layer:
    def __init__(self, neurons, activation):
        self.neurons = neurons
        self.activation = activation

class Perceptron:
    def __init__(self, layers, epoch, learning_rate):
        self.layers = layers
        self.epoch = epoch
        self.learning_rate = learning_rate
        
    def sigmoid(x):
        return 1/(1 + np.exp(-x))
    
    def dsigmoid(x):
        return Perceptron.sigmoid(x) * (1 - Perceptron.sigmoid(x))
    
    def relu(x):
         return np.where(x > 0, x, 0)
    
    def tahn(x):
        return (np.exp(2*x) - 1)/(np.exp(2*x) + 1) 
    
    def drelu(x):
        return np.where(x > 0, 1, 0)
    
    def dtahn(x):
        return 4/((np.exp(x) + np.exp(-x)) ** 2) 
    
    def func(x):
        return np.where(x > 0.5, 1, 0)
    
    def Weight(self, X, Y):
        W = []
        b = []
        w = np.array([np.random.uniform(-1, 1, (len(X.iloc[0]), self.layers[0].neurons))])
        w = w.reshape(len(X.iloc[0]), self.layers[0].neurons)
        b_i = np.array([np.random.uniform(-1, 1, (len(X.iloc[0]), self.layers[0].neurons))])
        b_i = b_i.reshape(len(X.iloc[0]), self.layers[0].neurons)
        W.append(w)
        b.append(b_i)
        for i in range(len(self.layers) - 1):
            w = np.array([np.random.uniform(-1, 1, (self.layers[i].neurons, self.layers[i+1].neurons))])
            w = w.reshape(self.layers[i].neurons, self.layers[i+1].neurons)
            W.append(w)
            b_i = np.array([np.random.uniform(-1, 1, (self.layers[i].neurons, self.layers[i+1].neurons))])
            b_i = w.reshape(self.layers[i].neurons, self.layers[i+1].neurons)
            b.append(b_i)
        return W, b
    
    def first_way(self, W, b, X, y):
        p = []
        out = np.array([])
        for sample in range(len(X)):
            W, b = Perceptron.Weight(self, X, y)
            p_i = []
            for i in range(len(self.layers)):
                if i == 0:
                    preAct = np.dot(X.iloc[sample, :], W[i] + b[i])
                    if self.layers[i].activation == 'sigmoid':
                        postAct = Perceptron.sigmoid(preAct)
                    elif self.layers[i].activation == 'relu':
                        postAct = Perceptron.relu(preAct)
                    else:
                        postAct = Perceptron.tahn(preAct)
                    p_i.append(postAct)
                elif i == len(self.layers) - 1:
                    preAct = np.dot(postAct, W[i]+ b[i])
                    if self.layers[i].activation == 'sigmoid':
                        postAct = Perceptron.sigmoid(preAct)
                    elif self.layers[i].activation == 'relu':
                        postAct = Perceptron.relu(preAct)
                    else:
                        postAct = Perceptron.tahn(preAct)
                    out = np.append(out, postAct)
                else:
                    preAct = np.dot(postAct, W[i]+ b[i])
                    if self.layers[i].activation == 'sigmoid':
                        postAct = Perceptron.sigmoid(preAct)
                    elif self.layers[i].activation == 'relu':
                        postAct = Perceptron.relu(preAct)
                    else:
                        postAct = Perceptron.tahn(preAct)
                    p_i.append(postAct)
            p.append(p_i)
        return out
    
    def fit(self, W, b, X, y, out):
        Q = 0
        w_q = W
        b_q = b
        
        for ep in range(self.epoch):
            for sample in range(len(X)):
                Q += ((np.linalg.norm(out[sample] - y[sample])**2) / 2)
            Q /= len(X)

            for i in range(len(W)):
                for j in range(len(W[i])):
                    for k in range(len(W[i][j])):
                        if self.layers[i].activation == 'sigmoid':
                            error = Q * Perceptron.dsigmoid(W[i][j][k])
                            error_ = Q * Perceptron.dsigmoid(b[i][j][k])
                        elif self.layers[i].activation == 'relu':
                            error = Q * Perceptron.drelu(W[i][j][k])
                            error_ = Q * Perceptron.drelu(b[i][j][k])
                        else:
                            error = Q * Perceptron.dtahn(W[i][j][k])
                            error_ = Q * Perceptron.dtahn(b[i][j][k])
                        w_q[i][j][k] = error * W[i][j][k]
                        b_q[i][j][k] = error_ * b[i][j][k]

            for i in range(len(W)):
                for j in range(len(W[i])):
                    for k in range(len(W[i][j])):
                        W[i][j][k] = W[i][j][k] - self.learning_rate * w_q[i][j][k]
                        b[i][j][k] = b[i][j][k] - self.learning_rate * b_q[i][j][k]
        return W, b
                
    def predict(self, X, W, b):
        y_pred = []
        y_predicted = []

        for sample in range(len(X)):
            for i in range(len(self.layers)):
                if i == 0:
                    preAct = np.dot(X.iloc[sample, :], W[i] + b[i])
                    if self.layers[i].activation == 'sigmoid':
                        postAct = Perceptron.sigmoid(preAct)
                    elif self.layers[i].activation == 'relu':
                        postAct = Perceptron.relu(preAct)
                    else:
                        postAct = Perceptron.tahn(preAct)
                elif i != len(self.layers) - 1:
                    preAct = np.dot(postAct, W[i] + b[i])
                    if self.layers[i].activation == 'sigmoid':
                        postAct = Perceptron.sigmoid(preAct)
                    elif self.layers[i].activation == 'relu':
                        postAct = Perceptron.relu(preAct)
                    else:
                        postAct = Perceptron.tahn(preAct)
                else:
                    preAct = np.dot(postAct, W[i] + b[i])
                    y_predicted.append(Perceptron.func(preAct))
        for i in range(len(y_predicted)):
            if (y_predicted[i] == 1):
                y_pred.append(1)
            else:
                y_pred.append(0)
        return y_pred
            

    def accuracy(self, y_true, y_pred):
        accuracy = np.sum(y_true == y_pred) / len(y_true)
        return accuracy
        
cat = Perceptron(layers = [Layer(neurons=3, activation='relu'),
                           Layer(neurons=2, activation='tahn'), Layer(neurons=1, activation='sigmoid')],
                 epoch=100, learning_rate=0.01)
W, b = cat.Weight(X_classification_train, y_classification_train)
out = cat.first_way(W, b, X_classification_train, y_classification_train)
y_classification_train_np = np.array([y_classification_train])
y_classification_train_np = y_classification_train_np.ravel()
W, b = cat.fit(W, b, X_classification_train, y_classification_train_np, out)
y_pred = cat.predict(X_classification_test, W, b)
print(cat.accuracy(y_classification_test, y_pred))

0.5397923875432526
